## Building a Basic Data Pipeline to ETL (Extract, Transform, Load) Data from a CSV File into a SQL Database.

### Project Overview

#### In this project, I am building a simple data pipeline that:

#### 1. Extracts data from a CSV file.
#### 2. Transforms the data (e.g., clean or preprocess).
#### 3. Loads the data into a MySQL database.


#### Prerequisite before running below cells:

#### 1. We need to create a new database or we can use any existing MySQL database
#### 2. Create table with schema according to the columns available in the Players CSV file
#### 3. The above steps can be implemented by running the ipl_players_table.sql file which includes the required code

In [1]:
#Installing Required libraries

#pip install pandas sqlalchemy mysql-connector-python

In [2]:
#Importing required libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from datetime import datetime
import json

In [3]:
#Configuration

# Load configuration from a file
with open('config.json', 'r') as file:
    config = json.load(file)

username = config['username']
password = config['password']
hostname = config['hostname']
port = config['port']
database = config['database']

csv_file = 'Data/IPLPlayers.csv'
database_url = 'mysql+mysqlconnector://'+username+':'+password+'@'+hostname+':'+str(port)+'/'+database

In [4]:
#For checking the database URL
#database_url

In [5]:
# Step 1: Extract
def extract_data(file_path):
    df = pd.read_csv(file_path)
    return df

In [6]:
# Step 2: Clean
def clean_data(df, temp):
    # Removing unwanted column
    column_to_remove = df.columns[temp]
    df = df.drop(columns=[column_to_remove])
    df.replace('', np.nan, inplace=True)
    # Remove rows with any NaN or null values
    df_cleaned = df.dropna()
    # Remove duplicate rows
    df_cleaned = df_cleaned.drop_duplicates()
    return df_cleaned

In [7]:
# Function to convert date format to SQL Date format
"""
def convert_date(date_str):
    input_format = '%d-%b-%y'
    output_format = '%Y-%m-%d'
    date_obj = datetime.strptime(date_str, input_format)
    return date_obj.strftime(output_format)
"""
def convert_date(date_str):
    input_format = '%d-%b-%y'
    output_format = '%Y-%m-%d'
    if isinstance(date_str, str):
        try:
            date_obj = datetime.strptime(date_str, input_format)
            return date_obj.strftime(output_format)
        except ValueError:
            # Handle the case where the date_str does not match the input_format
            return date_str
    return date_str  # Return the value unchanged if it's not a string

In [8]:
# Step 3: Transform
def transform_data(df):
    df['Batting_Hand'] = df['Batting_Hand'].str.replace("_"," ")
    df['DOB'] = df['DOB'].apply(convert_date)
    return df

In [9]:
# Step 4: Load
def load_data(df, db_url):
    engine = create_engine(db_url)
    df.to_sql('players', engine, if_exists='replace', index=False)
    print('\nData loaded into MySQL database successfully.')

In [10]:
# Main pipeline function
def main():
    df = extract_data(csv_file)
    print("Showing first 3 rows of the loaded data: \n")
    print(df.head(3))
    df = clean_data(df,7) #Column 7 is unnecessary
    print()
    print("Showing Data after removing column 6: \n")
    print(df.head(3))
    df = transform_data(df)
    print("\nShowing Data after Data transformation: \n")
    print(df.head(3))
    load_data(df, database_url)

if __name__ == "__main__":
    main()

Showing first 3 rows of the loaded data: 

   Player_Id  Player_Name        DOB Batting_Hand     Bowling_Skill  \
0          1   SC Ganguly   8-Jul-72    Left_Hand  Right-arm medium   
1          2  BB McCullum  27-Sep-81   Right_Hand  Right-arm medium   
2          3   RT Ponting  19-Dec-74   Right_Hand  Right-arm medium   

       Country  Is_Umpire  Unnamed: 7  
0        India          0         NaN  
1  New Zealand          0         NaN  
2    Australia          0         NaN  

Showing Data after removing column 6: 

   Player_Id  Player_Name        DOB Batting_Hand     Bowling_Skill  \
0          1   SC Ganguly   8-Jul-72    Left_Hand  Right-arm medium   
1          2  BB McCullum  27-Sep-81   Right_Hand  Right-arm medium   
2          3   RT Ponting  19-Dec-74   Right_Hand  Right-arm medium   

       Country  Is_Umpire  
0        India          0  
1  New Zealand          0  
2    Australia          0  

Showing Data after Data transformation: 

   Player_Id  Player_Name      

In [11]:
engine = create_engine(database_url)

# Use a text object to execute raw SQL
with engine.connect() as connection:
    # Define the raw SQL query using text()
    sql_query = text("SELECT * FROM players")
    
    # Execute the query
    result = connection.execute(sql_query)
    
    # Fetch the results
    for row in result:
        print(row)


(1, 'SC Ganguly', '1972-07-08', 'Left Hand', 'Right-arm medium', 'India', 0)
(2, 'BB McCullum', '1981-09-27', 'Right Hand', 'Right-arm medium', 'New Zealand', 0)
(3, 'RT Ponting', '1974-12-19', 'Right Hand', 'Right-arm medium', 'Australia', 0)
(4, 'DJ Hussey', '1977-07-15', 'Right Hand', 'Right-arm offbreak', 'Australia', 0)
(5, 'Mohammad Hafeez', '1980-10-17', 'Right Hand', 'Right-arm offbreak', 'Pakistan', 0)
(6, 'R Dravid', '1973-01-11', 'Right Hand', 'Right-arm offbreak', 'India', 0)
(7, 'W Jaffer', '1978-02-16', 'Right Hand', 'Right-arm offbreak', 'India', 0)
(8, 'V Kohli', '1988-11-05', 'Right Hand', 'Right-arm medium', 'India', 0)
(9, 'JH Kallis', '1975-10-16', 'Right Hand', 'Right-arm fast-medium', 'South Africa', 0)
(10, 'CL White', '1983-08-18', 'Right Hand', 'Legbreak googly', 'Australia', 0)
(11, 'MV Boucher', '1976-12-03', 'Right Hand', 'Right-arm medium', 'South Africa', 0)
(12, 'B Akhil', '1977-10-07', 'Right Hand', 'Right-arm medium-fast', 'India', 0)
(13, 'AA Noffke', 